# Assignment 2: Performance Metrics & First Week Flop/s

This assignment has some questions that you need to answer with text, and some code that you need to write.

You should put all of you textual answers in this notebook: `Insert->Insert Cell Below` to create a new cell below
the question, and `Cell->Cell Type->Markdown` to make it a cell for entering text.

You will test your code on the compute nodes of pace-ice, and that it also where we will evaluate it.
Please complete the text portions when you are logged into a head node working locally, and leave the compute nodes for when you actually need them.

**Due: Tuesday, September 2, 9:30 am**

**Total: 10 pts + 2 bonus pts (1 for working on a node with GPUs, 1 for optimizing the flop/s code)**

## Performance Metrics

In class we talked about the _strong-scaling efficiency_ of a parallel algorithm / machine pair: $H_f(P) = T_f(1) / (P T_f(P))$.

We then talked about the _weak-scaling efficiency_ of algorithm $f$ that can be applied to different problem sizes $N$: $E_f(N,P) = T_f(N/P,1) / T_f(N,P)$.

The question came up of how they are related to each other.

First, the notion of strong scaling doesn't have a concept of problem size, so let's add it: let's define

$$H_f(N,P) = T_f(N,1) / (P T_f(N,P)).$$

This is simply strong-scaling efficiency for each problem instance individually.

**Question 1 (1 pt):** Show that the relative order of strong and weak scaling efficiency (Whether $H_f(N,P) < E_f(N,P)$ or $E_f(N,P) < H_f(N,P)$) can be related to the efficiency of the serial algorithm, that is, whether $T_f(N,1)$ as a function of $N$ exhibits superlinear or sublinear behavior.

## PACE-ICE

**Head node exercise 1 (1 pt):** What command should you run from a head node to see a list of all the compute nodes in `coc-ice` and their availability? [Resource for this question: the [orientation slides](http://pace.gatech.edu/sites/default/files/pace-ice_orientation_2.pdf)]

###### queues
- coc-ice
- coc-ice-gpu
- coc-ice-multi
- coc-ice-long
- coc-ice-grade
- coc-ice-devel

qsub qdel showq qstat

In [1]:
# <-- Put your command there
pace-check-queue coc-ice


	** NEW FEATURE : add '-s' to pace-check-queue to list 
	** scheduler features for each node

=== coc-ice Queue Summary: ====
	Last Update                            : 08/30/2019 09:00:02
	Number of Nodes (Accepting Jobs/Total) : 48/49 (97.96%)
	Number of Cores (Used/Total)           : 1/916 ( 0.11%)
	Amount of Memory (Used/Total) (MB)     : 118809/8105865 ( 1.47%)
  Hostname       tasks/np Cpu%  loadav%  used/totmem(MB)   Mem%   Accepting Jobs? 
rich133-h35-15-r   1/28    3.6     0.8      2920/131126     2.2    Yes (free)             
rich133-h35-16-l   0/28    0.0     0.3      2885/131126     2.2    Yes (free)             
rich133-h35-16-r   0/28    0.0     1.0      2907/131126     2.2    Yes (free)             
rich133-h35-17-l   0/28    0.0     1.0      5037/518966     1.0    Yes (free)             
rich133-h35-17-r   0/28    0.0     0.2      5034/518966     1.0    Yes (free)             
rich133-h35-18-l   0/28    0.0     0.7      5063/518966     1.0    Yes (free)             
ri

Try it out: open up this notebook on a head node and compare the list you get to the [orientation slides](http://pace.gatech.edu/sites/default/files/pace-ice_orientation_2.pdf).  You'll see that it has grown, and they haven't updated the orientation slides.  We'll just have to find out what all these new nodes are for ourselves.

---
### A word on running jupyter on pace-ice:

As we discussed in class, screen refresh can be a bit laggy if you try to run a jupyter notebook through a browser opened on the head node or a compute node.  See the [guide](../../notes/logistics/compute-node-notebook.ipynb) in the notes for instructions on runnin the jupyter server on the compute nodes and the browser on your own computer.  You don't have to work directly in the notebook: you can work on you answers in the terminal, and then paste them into the notebook, as long as you're confident that they are correct.

---

**Head node exercise 2 (1 pt):** From the output of the above answer, you can probably see that we have a few different types of nodes to work with.  Fill in the blanks in the list below, describing the _properties_ of the different types. [Command line tools you might want to use: `qnodes`, `grep`, `sort`]

1. **24** nodes with **28** CPU core(s) and no GPUs
2. **12** nodes with **12** CPU core(s) and **2** GPU(s) of type NVIDIA Tesla **K40**.
3. **25** nodes with **8** CPU core(s) and **1** GPU(s) of type NVIDIA Tesla **P100** (this group includes `rich133-s42-21.pace.gatech.edu`, even though that the type of GPU is not listed for this node)
5. One node (`rich133-s30-20.pace.gatech.edu`) with **24** CPU core(s) (currently offline).

In [2]:
qnodes | grep properties | sort | wc -l

49


In [6]:
qnodes | grep -10 rich133-s42-21.pace.gatech.edu

     power_state = Running
     np = 12
     properties = core12,mhz2400,ib,ibQDR,localdisk,nvidiagpu,teslak40,ssd,5-2620v3,intel,rhel6
     ntype = cluster
     status = rectime=1567171958,macaddr=40:8d:5c:d4:eb:8b,cpuclock=Fixed,varattr=,jobs=,state=free,size=87592724kb:88117536kb,netload=3685577401,gres=,loadave=0.09,ncpus=12,physmem=132178344kb,availmem=132814684kb,totmem=134275492kb,idletime=1875434,nusers=0,nsessions=0,uname=Linux rich133-s30-22.pace.gatech.edu 2.6.32-573.12.1.el6.x86_64 #1 SMP Mon Nov 23 12:55:32 EST 2015 x86_64,opsys=RHEL6.7
     mom_service_port = 15002
     mom_manager_port = 15003
     gpus = 2
     gpu_status = gpu[1]=gpu_id=00000000:81:00.0;gpu_product_name=Tesla K40m;gpu_display=Disabled;gpu_pci_device_id=102310DE;gpu_pci_location_id=00000000:81:00.0;gpu_fan_speed=N/A;gpu_mode=Exclusive_Process;gpu_state=Unallocated;gpu_utilization=0 %;gpu_memory_utilization=0 %;gpu_ecc_mode=Enabled;gpu_single_bit_ecc_errors=1;gpu_double_bit_ecc_errors=1;gpu_temperature=2

**Head node exercise 3 (1 pt):** For the next questions, I need you to log in to compute nodes to find out about them, but you need to be able to specify which type of compute nodes you are accessing.

For each of the types of nodes 1, 2, and 3 in the question above, give me a `qsub` command to start a `jupyter_notebook_script.sh` job on that type of node, with the following requirements:

* The job should give you exclusive access to one node and all its cores and devices.
* The job should begin in the CSE6230 directory.
* The job should end after 30 minutes.

[Resources: [compute-node-notebook.ipynb](../../notes/logistics/compute-node-notebook.ipynb), [orientation slides](http://pace.gatech.edu/sites/default/files/pace-ice_orientation_2.pdf)]

In [4]:
man qsub

qsub(1B)                              PBS                             qsub(1B)



NAME
       qsub - submit pbs job

SYNOPSIS
       qsub   [-a   date_time]   [-A  account_string]  [-b  secs]  [-c  check-
       point_options] [-C directive_prefix] [-d path] [-D path] [-e path] [-f]
       [-h]  [-I] [-j join] [-k keep] [-l resource_list] [-m mail_options] [-M
       user_list] [-n node exclusive] [-N name] [-o path]  [-p  priority]  [-P
       proxy_username[:group]]   [-q  destination]  [-r  c] [-S path_list] [-t
       array_request] [-T prologue/epilogue script_name]  [-u  user_list]  [-v
       variable_list] [-V] [-w] path [-W additional_attributes] [-x] [-X] [-z]
       [script]

DESCRIPTION
       To create a job is to submit an executable script to  a  batch  server.
       The  batch  server  will  be the default server unless the -q option is
       specified.  See discussion of PBS_DEFAULT under  Environment  Variables
       below.   Typically, the script is a shell script

                      will supply the name of the host upon which it  is  exe-
                      cuting for the hostname component.

               hostname:path_name
                      Where  path_name  is not an absolute path name, then the
                      qsub command will not expand the path name  relative  to
                      the current working directory of the command.  On deliv-
                      ery of  the  standard  error,  the  path  name  will  be
                      expanded  relative  to  the user’s home directory on the
                      hostname system.

               path_name
                      Where path_name specifies an absolute  path  name,  then
                      the qsub will supply the name of the host on which it is
                      executing for the hostname

               hostname:path_name
                      Where path_name specifies an  absolute  path  name,  the
                      path will be used as speci

                      cuting for the hostname component.

               hostname:path_name
                      Where  path_name  is not an absolute path name, then the
                      qsub command will not expand the path name  relative  to
                      the current working directory of the command.  On deliv-
                      ery of the  standard  output,  the  path  name  will  be
                      expanded  relative  to  the user’s home directory on the
                      hostname system.

               path_name
                      Where path_name specifies an absolute  path  name,  then
                      the qsub will supply the name of the host on which it is
                      executing for the hostname

               hostname:path_name
                      Where path_name specifies an  absolute  path  name,  the
                      path will be used as specified.  hostname.

               hostname:
                      Where  hostnam

               string,  then the string must be enclosed with either single or
               double quote marks.

               PBS currently supports the following attributes within  the  -W
               option.

               depend=dependency_list
               Defines the dependency between this and other jobs.  The depen-
               dency_list is in the form:
               type[:argument[:argument...][,type:argument...].
               The argument is either a numeric count or a PBS job id  accord-
               ing  to type .  If argument is a count, it must be greater than
               0.  If it is a job id and  not  fully  specified  in  the  form
               seq_number.server.name,  it  will  be expanded according to the
               default server rules which apply to job IDs on  most  commands.
               If  argument  is  null  (the preceding colon need not be speci-
               fied), the dependency of  the  corresponding  type  is  cleared
      

               regardless of the direction of the copy.  The  name  local_file
               is  the  name of the file on the system where the job executed.
               It may be an absolute path or relative to the home directory of
               the  user.  The name remote_file is the destination name on the
               host specified by hostname.  The name may be absolute or  rela-
               tive to the user’s home directory on the destination host.  The
               use of wildcards in the file name is not recommended.  The file
               names  map to a remote copy program (rcp) call on the execution
               system in the follow manner:
               For stagein:   rcp hostname:remote_file local_file
               For stageout:  rcp local_file hostname:remote_file
               Data staging examples:
               -W stagein=/tmp/input.txt@headnode:/home/user/input.txt
               -W stageout=/tmp/output.txt@headnode:/home/user/output.txt
         


       If  an  option  is  present in a directive and not on the command line,
       that option and its argument, if any, will be processed as  if  it  had
       occurred on the command line.

       The  directive  prefix string will be determined in order of preference
       from:

           The value of the -C option argument if the option is  specified  on
           the command line.

           The value of the environment variable PBS_DPREFIX if it is defined.

           The four character string #PBS.

       If the -C option is found in a directive in the script file, it will be
       ignored.

       User Authorization:

       When  the  user submits a job from a system other than the one on which
       the PBS Server is running, the name under which the job is to  be  exe-
       cuted  is  selected  according to the rules listed under the -u option.
       The user submitting the job must be authorized to run the job under the
       execution user name.  This aut

In [ ]:
# put the qsub command for type 1 in this cell
qsub -d $CSE6230_DIR -q coc-ice -l nodes=1:ppn=4:gpus=2:teslak40:exclusive_process,walltime=00:30:00 $CSE6230_DIR/utils/jupyter_notebook_job.sh

In [ ]:
# put the qsub command for type 2 in this cell


In [ ]:
# put the qsub command for type 3 in this cell


## What have we got to work with?

Now, we need to switch from a notebook running on the head node to one running on a compue node, so `File->Save and Checkpoint` this notebook and `File->Close and Halt` it.  (Now would also be a good time to `git add` and `git commit` changes to this file.)  Use one of your ineractive job scripts to connect to a compute node and run the notebook there.
See you on the other side!

---

Okay, you're running on the compute node.

**Compute node exercise 1 (2 pt):** Using bash scripting (`awk`, `grep`, `sed`) or any other tool you like (you could, e.g., write a python script in a separate file and call it, as long as you `git add` it), set the variables in the cell below so that the printout that follows is correct.  You script should be correct on any type of compute node.

Resources: the file `/proc/cpuinfo`, the utility `nvidia-smi`; if you are very new to using a shell command line and the utilities that go with it in linux, please look at the [training slides on Linux](https://pace.gatech.edu/training) from PACE.

Note: when you run a command in backticks, you can assign its value to a variable like

```bash
MY_FILES=`ls -al`
```

Also note: when ever you encounter a new program or utility `AAA` that you've never used before, `man AAA` or `AAA --help` are the first places to go if you want to know what different command line flags do.

Some [one-liners](https://en.wikipedia.org/wiki/One-liner_program) that you may find useful:

* `grep -P -m 1 -o -e "(?<=XXX\s: ).*" YYY`: look in file `YYY` for the string "XXX   : " (with an arbitrary number of spaces between `XXX` and `:`) and print what comes after that on the line
* `wc -l YYY` counts the number of lines in file `YYY`
* most command line utilities that read files can also read the output of a previous command with a pipe `|`, for example to count the number of files in a directory:

```bash
ls -al | wc -l
```

* `grep -c "ZZZ" YYY` count the number of times the string `ZZZ` occurs in file `YYY`
* Nodes without GPUs won't have the `nvidia-smi` utility.  You can tell when a utility is unavailable if `which AAA` returns an error.  If you want to write a one-liner that only runs command `AAA` when `nvidia-smi` when it's available, you can do that like this:

```bash
(which nvidia-smi &> /dev/null) && (AAA)
```

In [11]:
head /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU E5-2620 v3 @ 2.40GHz
stepping	: 2
microcode	: 54
cpu MHz		: 2400.021
cache size	: 15360 KB
physical id	: 0


In [21]:
nvidia-smi -L | grep '0:' 

GPU 0: Tesla K40m (UUID: GPU-9264e1d1-674f-4941-800e-79f513dd4816)


In [16]:
CPU_NAME="Intel(R) Xeon(R) CPU E5-2620 v3 @ 2.40GHz"
CORE_COUNT="6"
GPU_NAME="Tesla K40m"
GPU_COUNT="2"

In [17]:
echo "This nodes has ${CORE_COUNT} cores: its architecture is (Manufacturer, Product Id) ${CPU_NAME}"
if [[ ! $GPU_COUNT || $GPU_COUNT == 0 ]] ;  then
    echo "This node has no GPUs"
else
    echo "This node has ${GPU_COUNT} GPUs: its/their architecture is (Manufacturer, Product Id) ${GPU_NAME}"
fi

This nodes has 6 cores: its architecture is (Manufacturer, Product Id) Intel(R) Xeon(R) CPU E5-2620 v3 @ 2.40GHz
This node has 2 GPUs: its/their architecture is (Manufacturer, Product Id) Tesla K40m


**Compute node exercise 2 (1 pt):** After you have logged out of the compute node, use whatever resources published on the web you can find to estimate the peak _single precision_ (aka FP32) flop/s of this node (you only need to do this step for one of the types of nodes, not all of them).

[Resources: [ark.intel.com](https://ark.intel.com), [wikipedia:FLOPS](https://en.wikipedia.org/wiki/FLOPS), [wikichip](https://en.wikichip.org), our notebook on [processors](../../notes/processors/processors-alone.ipynb)

I have chosen node type __ .  The peak flop/s for this node is ____ gigaflop/s.  Here is how I calculated that:

(calculation goes here)

## Flop/s fever

We've got to scratch that itch: we just want to go fast.  Okay, let's get it out of our system, and we'll look at more practical computations in future assignments.

You should choose one of the node types for this task.  Because this is more complex if multiple devices are involved
**1 bonus point** is earned for choosing a node with GPUs.

**Compute node exercise 3 (2 pts):** The command below will compile and runs essentially the following computation:

```C
for (i = 0; i < Nh; i++) { /* this loop will run on the "host" (CPUs) */
  for (j = 0; j < T; j++) {
    ah[i] = ah[i] * b + c;
  }
}

for (i = 0; i < Nd; i++) { /* this loop will run on the "device" (GPUs) */
  for (j = 0; j < T; j++) {
    ad[i] = ad[i] * b + c;
  }
}
```
And it will report the flop/s for the whole calculation.

`Nh` array entries will be on the host and `Nd` entries will be on each of the devices.  Try to find values of `Nh`, `Nd`, and `T`, and (optionally) compiler optimization flags that give you the highest flop/s.  Things to consider:

- Try to make your whole computation run for about a second.
- The time reported is the maximum time for any device: if one sits idle while the other finishes, it will rob you of flop/s.
- I suggest looking at one type of device at a time: set one of `Nh` or `Nd` to zero.  Once you've found your best flop/s for that device, optimize the other, and then try to strike a balance.
- Experiment with the merits of putting more weight on `Nh` and `Nd` vs more weight on `T`.
  Try to use **Little's Law** to make sure that you have enough parallelism to keep the pipelines filled.
- You can also choose to pass the option `Bs=X` to control the thread block size for the GPU, where `X` is a power of 2 between 64 and 2048.

In [18]:
make run_fma_prof Nh=256 Nd=256 T=256 COPTFLAGS='-O -xHost' CUOPTFLAGS='-O' # modify this for peak flop/s

icc -g -Wall -std=c99 -fPIC -O -xHost -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_prof.o fma_prof.c
icc -g -Wall -std=c99 -fPIC -O -xHost -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_omp.o fma_omp.c
icc -g -Wall -std=c99 -fPIC -O -xHost -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_loop_host.o fma_loop_host.c
nvcc -ccbin=icpc -Xcompiler '-fPIC' -O -dc -o fma_cuda.o fma_cuda.cu
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc -ccbin=icpc -Xcompiler '-fPIC' -O -dc -o fma_loop_dev.o fma_loop_dev.cu
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc -ccbin=icpc -Xcompiler '-fPIC' -dlink  fma_cuda.o fma_loop_dev.o -o fma_cuda_link.o
nvcc warning : The 'compute_20', 'sm_20', an

**Compute Node Exercise 4 (Bonus 1 pt):** Now let's see if we can make any transformations to the code to make a difference.

We will run the same program, but with fused multiply add loops that you have tried to optimize.  You should edit the files
`fma_loop_host_opt.cu` and/or `fma_loop_dev_opt.c`: they start out exactly the same as the reference implementations used above.

In [19]:
cat fma_loop_host_opt.c


#include "fma_host.h"

/* fma_loop: Fused Multiply Add loop
 *           -     -        -
 *
 * a[:] = a[:] * b + c, T times
 *
 * Inputs:
 * N : the size of the array
 * T : the number of loops
 * b : the multiplier
 * c : the shift
 *
 * Input-Outputs:
 * a : the array
 */
void
fma_loop_host (int N, int T, float *a, float b, float c)
{
  for (int i = 0; i < N; i++) {
    for (int j = 0; j < T; j++) {
      a[i] = a[i] * b + c;
    }
  }
}


In [20]:
cat fma_loop_dev_opt.cu


#include "fma_dev.h"

__global__
void
fma_loop_dev (int N, int T, float *a, float b, float c)
{
  int my_thread = threadIdx.x + blockIdx.x * blockDim.x;
  int num_threads = gridDim.x * blockDim.x;

  for (int i = my_thread; i < N; i+= num_threads) {
    for (int j = 0; j < T; j++) {
      a[i] = a[i] * b + c;
    }
  }
}


In [ ]:
diff fma_loop_host.c fma_loop_host_opt.c

In [ ]:
diff fma_loop_dev.cu fma_loop_dev_opt.cu

See if you can exploit vectorization, instruction level parallelism, and/or loop transformations to get a boost.

In [ ]:
make run_fma_prof_opt Nh=256 Nd=256 T=256 COPTFLAGS='-O -xHost' CUOPTFLAGS='-O' # modify this for peak flop/s

## Submitting this work

**Workstation exercise 1 (1 pt):** When you have completed the rest of this assignment, `git add` the changes to this file, the source files you modified, and any scripts you added, and `git commit` them.  Having commited your changes, you should `git push` them to the private repository that you have on `github.gatech.edu`.

Our TA Han Sol Suh will email each of you a individualized [deploy key](https://developer.github.com/v3/guides/managing-deploy-keys/) that will allow him to read the contents of your repository.  

**Assignments need to be formally submitted to canvas,** but the totality of your submission on canvas should be a git revision hash or branch name indicating the version of your repository we should use to grade the assignment.